<a href="https://colab.research.google.com/github/aTom-Pie/DW_Matrix_Week_3/blob/master/Matrix3Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd '/content/drive/My Drive/Colab Notebooks/Matrix3'

/content/drive/My Drive/Colab Notebooks/Matrix3


In [2]:
ls

 data/                      Matrix3_Day1.ipynb   Matrix3_Day4.ipynb
'Kopia Matrix3Day5.ipynb'   Matrix3_Day2.ipynb   Matrix3Day5-2.ipynb
 logs/                      Matrix3_Day3.ipynb   Matrix3Day5.ipynb


In [0]:
# !pip install hyperopt

In [0]:
import os
import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from skimage import color, exposure

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Flatten,Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


# from collections import Counter

In [0]:
df=pd.read_csv('data/signnames.csv')
labels = df.to_dict()['b']

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

[X_train, y_train] = train['features'], train['labels']
[X_test, y_test] = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),  
      epochs=params_fit.get('epochs', 5), 
      verbose=params_fit.get('verbose', 1),  
      validation_data=params_fit.get('validation_data', (X_train, y_train)), 
      callbacks=[tensorboard_callback]    
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                     
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(num_classes, activation='softmax')

  ])
  

In [11]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.5367 - accuracy: 0.3056 - val_loss: 0.8376 - val_accuracy: 0.7417
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.7218 - accuracy: 0.7722 - val_loss: 0.1437 - val_accuracy: 0.9584
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3413 - accuracy: 0.8959 - val_loss: 0.0745 - val_accuracy: 0.9808
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2148 - accuracy: 0.9350 - val_loss: 0.0408 - val_accuracy: 0.9878
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1672 - accuracy: 0.9502 - val_loss: 0.0234 - val_accuracy: 0.9934


0.9687074829931973

In [12]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1239 - accuracy: 0.9687


[0.12394881248474121, 0.9687075018882751]

In [0]:
def get_model(params):
  return Sequential([
                     
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_1']),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_2']),
    
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_3']),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_1']),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_2']),
    
    Dense(num_classes, activation='softmax')

  ])
  

In [0]:
def func_obj(params):
  get_model(params)
  model.compile(loss='categorical_crossentropy', 
                optimizer='Adam', 
                metrics=['accuracy']
                )
  
  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)), 
      # epochs=7,
      epochs = int(params.get('epochs', 7)),   
      verbose=0,          
  )

  score = model.evaluate(X_test, y_test,verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [15]:
space = {
      'batch_size': hp.quniform('batch_size', 100, 200, 10),
      'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0.3, 0.5),
      'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0.3, 0.5),
      'dropout_cnn_block_3': hp.uniform('dropout_cnn_block_3', 0.3, 0.5),
      'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0.3, 0.7),
      'dropout_dense_block_2': hp.uniform('dropout_dense_block_2', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 120.0, 'dropout_cnn_block_1': 0.3537223943196351, 'dropout_cnn_block_2': 0.4228291979241502, 'dropout_cnn_block_3': 0.36067627286637355, 'dropout_dense_block_1': 0.438954637560455, 'dropout_dense_block_2': 0.5721150131322976}
accuracy=0.9800453782081604
{'batch_size': 120.0, 'dropout_cnn_block_1': 0.45142380003575777, 'dropout_cnn_block_2': 0.3789497726951591, 'dropout_cnn_block_3': 0.37209966656132293, 'dropout_dense_block_1': 0.32837828345420506, 'dropout_dense_block_2': 0.3419139026919166}
accuracy=0.9841269850730896
{'batch_size': 190.0, 'dropout_cnn_block_1': 0.3956649522287362, 'dropout_cnn_block_2': 0.46267563013910407, 'dropout_cnn_block_3': 0.44083456913354424, 'dropout_dense_block_1': 0.3384536135271977, 'dropout_dense_block_2': 0.5284022811903304}
accuracy=0.981632649898529
{'batch_size': 110.0, 'dropout_cnn_block_1': 0.4569961293643075, 'dropout_cnn_block_2': 0.41366045642562965, 'dropout_cnn_block_3': 0.4748603822167156, 'dropout_dense_block_1': 0.6499922493